In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import json
import os

## Experiments and Functions

In [2]:
experiments = [
            {'id': '1', 'source':'imdb', 'target':'uwcse', 'predicate':'workedunder', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '2', 'source':'uwcse', 'target':'imdb', 'predicate':'advisedby', 'to_predicate':'workedunder', 'arity': 2},
            {'id': '3', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'samevenue', 'arity': 2},
            #{'id': '4', 'source':'cora', 'target':'imdb', 'predicate':'samevenue', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '5', 'source':'cora', 'target':'imdb', 'predicate':'sametitle', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '6', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'sametitle', 'arity': 2},
            ##{'id': '5', 'source':'uwcse', 'target':'cora', 'predicate':'advisedby', 'to_predicate':'samevenue', 'arity': 2},
            ##{'id': '6', 'source':'cora', 'target':'uwcse', 'predicate':'samevenue', 'to_predicate':'advisedby', 'arity': 2},
            {'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2},
            #{'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2},
            {'id': '9', 'source':'nell_sports', 'target':'nell_finances', 'predicate':'teamplayssport', 'to_predicate':'companyeconomicsector', 'arity': 2},
            {'id': '10', 'source':'nell_finances', 'target':'nell_sports', 'predicate':'companyeconomicsector', 'to_predicate':'teamplayssport', 'arity': 2},
]

In [3]:
def load_data(experiments, metric, directory):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(metric != ''):
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        else:
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '.json'
        try:
            with open(file_path, 'r') as fp:
                results = json.load(fp)

                data[experiment_title] = results
        except:
            continue
    return data

In [4]:
def converte_dataframe_latex(dt):
    for j in range(len(experiments)):
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']

## Estudo dos Mapeamentos

### Depth-First Mapping

In [5]:
# Setting paths parameters
mapping_order = 'comites'
k = 1
compare_stopwords = False

#Removing Yeast and Twitter pair of experiments because there is no stopwords in predicates
#exp = experiments.copy()
#exp.remove({'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2})
#exp.remove({'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2})

In [9]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/'
#data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
#data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
#data_wmd_stopwords = load_data(experiments, 'wmd', path)
#data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

data_majority_stopwords = load_data(experiments, 'majority_vote', path)
data_borda_stopwords = load_data(experiments, 'borda_count', path)

if compare_stopwords:
    # Removing stopwords
    path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
    data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
    data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
    data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
    data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

treeboostler_path = f'/results/baselines/transfer/'
#treeboostler_data = load_data(experiments, '', treeboostler_path)
#rdnb_data = load_data(experiments, '', treeboostler_path + '/rdn_b/')

to_latex = {}

for metric in ['CLL', 'AUC ROC', 'AUC PR', 'Learning time', 'Mapping time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    #display(Markdown('# Results for ' + metric))
    table = []
    for j in range(len(experiments)):
            
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_majority_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}
            
            if metric == 'Mapping time':
                #results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
                #results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
                #results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
                #results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

                results_using_stopwords['majority_vote'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_majority_stopwords[str(dataset)]])
                results_using_stopwords['borda_count'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_borda_stopwords[str(dataset)]])

            else:
                # Results using Stopwords
                #results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
                #results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
                #results_using_stopwords['wmd'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
                #results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

                results_using_stopwords['majority_vote'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_majority_stopwords[str(dataset)]])
                results_using_stopwords['borda_count'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_borda_stopwords[str(dataset)]])
                
            #results_using_stopwords['softcosine no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            #results_using_stopwords['euclidean no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            #results_using_stopwords['wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            #results_using_stopwords['relax-wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            if compare_stopwords:
                # Results removing stopwords
                results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
                results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
                results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
                results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])

            #if(experiments[j]['id'] != '10'):
                #treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            #rdnb = np.array([(np.array([item['rdn-b'][metric] for item in sublist])).mean() for sublist in rdnb_data[str(dataset)]])
            
#             if(experiments[j]['id'] != '10'):
#                 table.append([dataset,
#                 #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
#                 #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
#                              ])
#             else:
#                 table.append([dataset,
#                 #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
#                 #'NaN +/- NaN',
#                 '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
#               ])

            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                #'%.3f' % (rdnb.mean()),
                #'%.3f' % (treeboostler.mean()),
                #'%.3f ' % (results_using_stopwords['softcosine'].mean()),
                #'%.3f ' % (results_using_stopwords['euclidean'].mean()),
                #'%.3f ' % (results_using_stopwords['wmd'].mean()),
                #'%.3f ' % (results_using_stopwords['relax-wmd'].mean()),
                              
                '%.3f ' % (results_using_stopwords['majority_vote'].mean()),
                '%.3f ' % (results_using_stopwords['borda_count'].mean()),
                             ])
            else:
                table.append([dataset,
                #'%.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'NaN',
                #'%.3f' % (results_using_stopwords['softcosine'].mean()),
                #'%.3f' % (results_using_stopwords['euclidean'].mean()),
                #'%.3f' % (results_using_stopwords['wmd'].mean()),
                #'%.3f' % (results_using_stopwords['relax-wmd'].mean()),
                              
                '%.3f ' % (results_using_stopwords['majority_vote'].mean()),
                '%.3f ' % (results_using_stopwords['borda_count'].mean()),
              ])
            
            #table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10' and compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),              
                #'%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                #'%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                #'%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                #'%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                              
                '%.3f +/- %.3f' % (results_removing_stopwords['majority_vote'].mean(), 2 * results_removing_stopwords['majority_vote'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['borda_count'].mean(), 2 * results_removing_stopwords['borda_count'].std()),    
                             ])
            elif(compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                             ])

            
            # No Revision
            #table.append([dataset + '_NR',
            #    '%.3f +/- %.3f' % (results_using_stopwords['softcosine no revision'].mean(), 2 * results_using_stopwords['softcosine'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['euclidean no revision'].mean(), 2 * results_using_stopwords['euclidean'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['wmd no revision'].mean(), 2 * results_using_stopwords['wmd'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd no revision'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
            #    '%.3f +/- %.3f' % (treeboostler_no_revision.mean(), 2 * treeboostler_no_revision.std()),
            #    '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
            #])
            #table.append(['', '', '', '', '', '', ''])

    #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
   #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
    #dt = pd.DataFrame(table, columns=['Experiment', 'Transfer Learning - Soft Cosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - RWMD']) #, 'Learning from scratch (RDN-B)']))
    dt = pd.DataFrame(table, columns=['Experiment', 'Transfer Learning - Majority', 'Transfer Learning - Borda'])
    #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

    for j in range(len(experiments)):
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        
        if dataset not in to_latex:
            to_latex[dataset] = {}
        try:
            to_latex[dataset][metric] = dt.set_index('Experiment').T.reset_index()[dataset].to_list()
        except:
            pass
        
for experiment in to_latex:
    if to_latex[experiment]:
        display(Markdown('# Results for ' + experiment))
        #to_latex[experiment][''] = ['TransBoostler Soft Cosine', 'TransBoostler Euclidean', 'TransBoostler WMD', 'TransBoostler RWMD']
        to_latex[experiment][''] = ['TransBoostler Majority', 'TransBoostler Borda Count']
        print(pd.DataFrame(to_latex[experiment], columns=['', 'CLL', 'AUC ROC', 'AUC PR', 'Learning time', 'Mapping time']).to_latex(index=False))

# Results for 1_imdb_uwcse

\begin{tabular}{llllll}
\toprule
                          &     CLL & AUC ROC & AUC PR & Learning time & Mapping time \\
\midrule
   TransBoostler Majority & -0.254  &  0.938  & 0.270  &        6.134  &       0.037  \\
TransBoostler Borda Count & -0.252  &  0.934  & 0.273  &       27.401  &      21.370  \\
\bottomrule
\end{tabular}



# Results for 3_imdb_cora

\begin{tabular}{llllll}
\toprule
                          &     CLL & AUC ROC & AUC PR & Learning time & Mapping time \\
\midrule
   TransBoostler Majority & -0.678  &  0.589  & 0.453  &       72.344  &      20.970  \\
TransBoostler Borda Count & -0.673  &  0.599  & 0.462  &       71.718  &      20.968  \\
\bottomrule
\end{tabular}



# Results for 7_yeast_twitter

\begin{tabular}{llllll}
\toprule
                          &     CLL & AUC ROC & AUC PR & Learning time & Mapping time \\
\midrule
   TransBoostler Majority & -0.102  &  0.994  & 0.367  &       70.755  &      27.651  \\
TransBoostler Borda Count & -0.088  &  0.995  & 0.459  &       98.872  &      27.647  \\
\bottomrule
\end{tabular}



# Results for 9_nell_sports_nell_finances

\begin{tabular}{llllll}
\toprule
                          &     CLL & AUC ROC & AUC PR & Learning time & Mapping time \\
\midrule
   TransBoostler Majority & -0.318  &  0.753  & 0.077  &       51.716  &       0.178  \\
TransBoostler Borda Count & -0.329  &  0.720  & 0.069  &       87.488  &      39.271  \\
\bottomrule
\end{tabular}



# Results for 10_nell_finances_nell_sports

\begin{tabular}{llllll}
\toprule
                          &     CLL & AUC ROC & AUC PR & Learning time & Mapping time \\
\midrule
   TransBoostler Majority & -0.085  &  0.993  & 0.291  &      276.388  &      42.718  \\
TransBoostler Borda Count & -0.147  &  0.991  & 0.219  &       80.204  &       0.165  \\
\bottomrule
\end{tabular}



### Ranked-First Mapping

In [8]:
# Setting paths parameters
mapping_order = 'most_similar'
k = '2'
method = f'k_{k}/'
#method='hungaro/'

compare_stopwords = False

In [10]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/w_stopwords/{method}'
data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

if compare_stopwords:
    # Removing stopwords
    path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
    data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
    data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
    data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
    data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

treeboostler_path = f'/results/baselines/transfer/'
treeboostler_data = load_data(experiments, '', treeboostler_path)
rdnb_data = load_data(experiments, '', treeboostler_path + '/rdn_b/')

to_latex = {}

for metric in ['CLL', 'AUC ROC', 'AUC PR', 'Learning time', 'Mapping time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    #display(Markdown('# Results for ' + metric))
    table = []
    for j in range(len(experiments)):
            
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_wmd_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}
            
            if metric == 'Mapping time':
                results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
                results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
                results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
                results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])
            else:
                # Results using Stopwords
                results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
                results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
                results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
                results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            #results_using_stopwords['softcosine no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            #results_using_stopwords['euclidean no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            #results_using_stopwords['wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            #results_using_stopwords['relax-wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            if compare_stopwords:
                # Results removing stopwords
                results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
                results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
                results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
                results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])

            #if(experiments[j]['id'] != '10'):
                #treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            #rdnb = np.array([(np.array([item['rdn-b'][metric] for item in sublist])).mean() for sublist in rdnb_data[str(dataset)]])
            
                        #if(experiments[j]['id'] != '10'):
                #treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            #rdnb = np.array([(np.array([item['rdn-b'][metric] for item in sublist])).mean() for sublist in rdnb_data[str(dataset)]])
            
#             if(experiments[j]['id'] != '10'):
#                 table.append([dataset,
#                 #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
#                 #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
#                              ])
#             else:
#                 table.append([dataset,
#                 #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
#                 #'NaN +/- NaN',
#                 '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
#                 '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
#               ])

            

            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                #'%.3f' % (rdnb.mean()),
                #'%.3f' % (treeboostler.mean()),
                '%.3f ' % (results_using_stopwords['softcosine'].mean()),
                '%.3f ' % (results_using_stopwords['euclidean'].mean()),
                '%.3f ' % (results_using_stopwords['wmd'].mean()),
                '%.3f ' % (results_using_stopwords['relax-wmd'].mean()),
                             ])
            else:
                table.append([dataset,
                #'%.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'NaN',
                '%.3f' % (results_using_stopwords['softcosine'].mean()),
                '%.3f' % (results_using_stopwords['euclidean'].mean()),
                '%.3f' % (results_using_stopwords['wmd'].mean()),
                '%.3f' % (results_using_stopwords['relax-wmd'].mean()),
              ])
            
            #table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10' and compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),              
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),                
                             ])
            elif(compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                             ])
            
            #table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10' and compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),              
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),                
                             ])
            elif(compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                             ])
            
            # No Revision
            #table.append([dataset + '_NR',
            #    '%.3f +/- %.3f' % (results_using_stopwords['softcosine no revision'].mean(), 2 * results_using_stopwords['softcosine'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['euclidean no revision'].mean(), 2 * results_using_stopwords['euclidean'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['wmd no revision'].mean(), 2 * results_using_stopwords['wmd'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd no revision'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
            #    '%.3f +/- %.3f' % (treeboostler_no_revision.mean(), 2 * treeboostler_no_revision.std()),
            #    '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
            #])
            #table.append(['', '', '', '', '', '', ''])

    #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
    dt = pd.DataFrame(table, columns=['Experiment', 'Transfer Learning - Soft Cosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - RWMD']) #, 'Learning from scratch (RDN-B)']))
    #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

    for j in range(len(experiments)):
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        
        if dataset not in to_latex:
            to_latex[dataset] = {}
        try:
            to_latex[dataset][metric] = dt.set_index('Experiment').T.reset_index()[dataset].to_list()
        except:
            pass
        
for experiment in to_latex:
    if to_latex[experiment]:
        display(Markdown('# Results for ' + experiment))
        to_latex[experiment][''] = ['TransBoostler Soft Cosine', 'TransBoostler Euclidean', 'TransBoostler WMD', 'TransBoostler RWMD']
        print(pd.DataFrame(to_latex[experiment], columns=['', 'CLL', 'AUC ROC', 'AUC PR', 'Learning time']).to_latex(index=False))

# Results for 7_yeast_twitter

\begin{tabular}{lllll}
\toprule
                          &     CLL & AUC ROC & AUC PR & Learning time \\
\midrule
TransBoostler Soft Cosine & -0.103  &  0.994  & 0.352  &       48.894  \\
  TransBoostler Euclidean & -0.105  &  0.994  & 0.386  &       49.004  \\
        TransBoostler WMD & -0.101  &  0.994  & 0.399  &       49.680  \\
       TransBoostler RWMD & -0.106  &  0.993  & 0.353  &       51.108  \\
\bottomrule
\end{tabular}

